In this recipe, we're going to cover how to compute phylogenetic measures of biological diversity using scikit-bio and pandas. 

The theory behind these metrics, as well as a comparison to non-phylogenetic measures of biological diversity is covered in detail in the [*Studying Biological Diversity* chapter](http://nbviewer.ipython.org/github/gregcaporaso/An-Introduction-To-Applied-Bioinformatics/blob/master/applications/1-biological-diversity.ipynb) of [*An Introduction to Applied Bioinformatics*](http://applied-bioinformatics.org). *An Introduction to Applied Bioinformatics*, or *IAB*, is an open access (Creative Commons-licensed) executable introductory bioinformatics text which pulls a lot of functionality from scikit-bio. 

Because the theory of phylogenetic diversity metrics is covered in detail in IAB, we won't discuss a lot of that here. Instead we'll focus on implementation.

## Configure the environment and prepare some data to work with

In this notebook we're going to use scikit-bio, numpy and pandas to perform alpha (within-sample) and beta (between-sample) diversity calculations.

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
from skbio.tree import TreeNode

Imagine we have three samples and nine *biological observations*. Think of these *biological observations* as species and the samples as 1 meter plots of land. If we compile a count of how many times each *biological observation* is observed in each sample (a *count vector*), we get a picture of the species composition of the given plot of land. Count vectors for multiple samples can be represented as a *sample by observation count matrix*.

A [BIOM table](http://biom-format.org/) is frequently used to represent sample by observation count matrices, but since BIOM is not a dependency of scikit-bio (and therefore not a dependency of the scikit-bio cookbook), and because we're only accessing this data in a very basic way, we're going to use a ``pandas.DataFrame`` object to store this data. When the ``biom_format.Table`` object is ported to scikit-bio (see [issue #489](https://github.com/biocore/scikit-bio/issues/489)), we'll update to use that here.

In [2]:
sample_ids = ['sample1', 'sample2', 'sample3']
observation_ids = ['species1', 'species2', 'species3', 
                   'species4', 'species5', 'species6',
                   'species7', 'species8', 'species9',]
data = np.array([[1, 1, 5],
                 [1, 2, 0],
                 [3, 1, 0],
                 [0, 2, 0],
                 [0, 0, 0],
                 [0, 0, 3],
                 [0, 0, 1],
                 [1, 0, 1],
                 [0, 0, 0]])

table1 = pd.DataFrame(data, index=observation_ids, columns=sample_ids)

table1

sample1  sample2  sample3
species1        1        1        5
species2        1        2        0
species3        3        1        0
species4        0        2        0
species5        0        0        0
species6        0        0        3
species7        0        0        1
species8        1        0        1
species9        0        0        0

It's then simple to access the counts of a particular species in a particular sample. For example, ``species3`` was observed three times in ``sample1``, one time in ``sample2``, and zero times in ``sample3``.

In [3]:
print(table1['sample1']['species3'])
print(table1['sample2']['species3'])
print(table1['sample3']['species3'])

3
1
0


Because we're going to perform phylogenetic diversity calculations here, we need a phylogenetic tree that represents the evolutionary relationships between the species in our table. (If you don't understand why that is, refer to *An Introduction to Applied Bioinformatics*.) We'll define our tree as a newick-formatted string, and then load that into a scikit-bio [``TreeNode`` object](http://scikit-bio.org/docs/latest/generated/skbio.tree.TreeNode.html#skbio.tree.TreeNode).

In [4]:
from io import StringIO
tree_s = StringIO(u'(((species1:0.2,species2:0.3):0.3,((species3:0.5,species4:0.3):0.2,species5:0.9):0.3):0.35,(((species6:0.2,species7:0.3):0.3,(species8:0.3,species9:0.4):0.7):0.2))root;')
tree = TreeNode.read(tree_s)

Here's what that tree looks like if we visualize it with [Archaeopteryx](https://sites.google.com/site/cmzmasek/home/software/archaeopteryx). At the left-most point on each branch, the length of that branch is indicated. Note that our observation ids are the tips in the tree.

<img src="assets/phylogenetic-diversity-tree.png">

## Computing Faith's Phylogenetic Diversity

The first metric that we'll look at is Faith's Phyogenetic Diversity, or PD. This was initially described in [Faith (1992)](http://www.sciencedirect.com/science/article/pii/0006320792912013), and is discussed in [IAB](http://nbviewer.ipython.org/github/gregcaporaso/An-Introduction-To-Applied-Bioinformatics/blob/master/applications/1-biological-diversity.ipynb).

The key piece of information that we need to compute PD is what nodes in a phylogenetic tree were observed in a given sample. From that, we can compute the total amount of branch length observed in that sample. Note that a value of PD will be specific to a given phylogenetic tree (e.g., if you multipled all of the branch lengths in the tree by 2, you'd double the PD).

Here we'll define a function that returns the set of nodes that are observed in a given tree, given a vector of species (or more generally *biological observations*) that were observed. The species represent the *tips* in our tree, so internally we refer to them as *tips*. 

This function gives us the set of nodes in the tree that would be observed given some ``observed_tips``. Most of these will be internal nodes (and thus generally unnamed). These are identified by tracing from each of the observed species (tips) back to the root of the tree.

Note that we've built a ``verbose`` flag into this function, which is useful for printing the previously unobserved branch lengths between each observed tip and the root. This is ``False`` by default (because most of the time it's annyoing to have extraneous information printed out by functions), but it can be useful for learning about how an algorithm works.

In [5]:
def get_observed_nodes(tree, observed_tips, verbose=False):
    # this function should be ported to the TreeNode object
    observed_nodes = set()
    # iterate over the observed tips in the tree
    for tip in observed_tips:
        t = tree.find(tip)
        observed_nodes.add(t)
        if verbose:
            print(t.name)
            print(t.length)
        for internal_node in t.ancestors():
            if internal_node.length is None:
                # we've hit the root
                pass
            else:
                if verbose and internal_node not in observed_nodes:
                    print(internal_node.length),
                observed_nodes.add(internal_node)
    return observed_nodes

In [6]:
print(get_observed_nodes(tree, ['species1']))

set([<TreeNode, name: unnamed, internal node count: 0, tips count: 2>, <TreeNode, name: species1, internal node count: 0, tips count: 0>, <TreeNode, name: unnamed, internal node count: 3, tips count: 5>])


In [7]:
print(get_observed_nodes(tree, ['species1', 'species3']))

set([<TreeNode, name: unnamed, internal node count: 0, tips count: 2>, <TreeNode, name: species3, internal node count: 0, tips count: 0>, <TreeNode, name: unnamed, internal node count: 3, tips count: 5>, <TreeNode, name: unnamed, internal node count: 0, tips count: 2>, <TreeNode, name: species1, internal node count: 0, tips count: 0>, <TreeNode, name: unnamed, internal node count: 1, tips count: 3>])


In [8]:
print(get_observed_nodes(tree, ['species3', 'species9']))

set([<TreeNode, name: unnamed, internal node count: 0, tips count: 2>, <TreeNode, name: species3, internal node count: 0, tips count: 0>, <TreeNode, name: unnamed, internal node count: 0, tips count: 2>, <TreeNode, name: unnamed, internal node count: 3, tips count: 5>, <TreeNode, name: unnamed, internal node count: 2, tips count: 4>, <TreeNode, name: species9, internal node count: 0, tips count: 0>, <TreeNode, name: unnamed, internal node count: 1, tips count: 3>])


Computing phylogenetic diversity is then simple. We just need to sum the branch lengths associated with the observed nodes. We'll define this as a function that takes a table and a sample id, and internally generates the list of observed tips/species.

We can apply this to compute the phylogenetic diversity for each of our samples. When running with ``verbose=True``, we can see the branch lengths that go into each computation. Only the branch lengths that haven't previously been observed are printed for an observed tip.

In [9]:
def get_observed_tips(table, sample_id):
    if sample_id not in table.columns:
        raise KeyError("sample_id (%s) is not present in table." % sample_id)
    return [obs_id for obs_id in table.index if table[sample_id][obs_id] > 0]

def phylogenetic_diversity(tree, table, sample_id, verbose=False):
    observed_tips = get_observed_tips(table, sample_id)
    observed_nodes = get_observed_nodes(tree, observed_tips, verbose=verbose)
    pd = sum(o.length for o in observed_nodes)
    return pd

In [10]:
pd1 = phylogenetic_diversity(tree, table1, 'sample1', verbose=True)
print(pd1)

species1
0.2
0.3
0.35
species2
0.3
species3
0.5
0.2
0.3
species8
0.3
0.7
0.2
3.35


In [11]:
pd2 = phylogenetic_diversity(tree, table1, 'sample2', verbose=True)
print(pd2)

species1
0.2
0.3
0.35
species2
0.3
species3
0.5
0.2
0.3
species4
0.3
2.45


In [12]:
pd3 = phylogenetic_diversity(tree, table1, 'sample3', verbose=True)
print(pd3)

species1
0.2
0.3
0.35
species6
0.2
0.3
0.2
species7
0.3
species8
0.3
0.7
2.85


## Computing Unweighted UniFrac

Unweighted UniFrac was introduced initially introduced in [Lozupone and Knight (2005)](http://aem.asm.org/content/71/12/8228.abstract), and provides a way to compute the dissimilarity of a pair of samples. Unweighted UniFrac is discussed in detail in [IAB](http://nbviewer.ipython.org/github/gregcaporaso/An-Introduction-To-Applied-Bioinformatics/blob/master/applications/1-biological-diversity.ipynb), so here we'll just focus on implementation. 

Unweighted UniFrac is closely related to PD. What we want to do here is divide the branch length that is observed in only one sample (the *unique branch length*) by the branch length observed in at least one of the samples (the *total branch length*). 

Since we've already defined a function that gives us the set of nodes that are observed in a sample, it's simple to compute that for each of our samples. We can then apply set operations to find the nodes observed in both samples (the union of the sets of observed nodes), and the nodes observed in either of the samples (the intersection of the sets of observed nodes). The set of nodes observed in only one sample (the unique nodes) is the set of nodes observed in either sample minus the set of nodes observed in both samples.

We can then get the branch length represented by the set of unique nodes and by the set of observed nodes, and compute unweighted UniFrac.

In [13]:
def unweighted_unifrac(tree, table, sample_id1, sample_id2, verbose=False):
    observed_tips1 = observed_tips = get_observed_tips(table, sample_id1)
    observed_tips2 = observed_tips = get_observed_tips(table, sample_id2)
    observed_nodes1 = get_observed_nodes(tree, observed_tips1, verbose=verbose)
    observed_nodes2 = get_observed_nodes(tree, observed_tips2, verbose=verbose)
    
    observed_branch_length = sum(o.length for o in observed_nodes1 | observed_nodes2)
    shared_branch_length = sum(o.length for o in observed_nodes1 & observed_nodes2)
    unique_branch_length = observed_branch_length - shared_branch_length
    unweighted_unifrac = unique_branch_length / observed_branch_length
    return unweighted_unifrac

We can now apply this to all pairs of samples to figure out which are more and less similar to each other. This tells us that samples 1 and 2 are the most similar (they have the lowest dissimilarity) and samples 2 and 3 are the least similar to each other (they have the highest dissimilarity).

In [14]:
unweighted_unifrac(tree, table1, 'sample1', 'sample2')

0.41095890410958896

In [15]:
unweighted_unifrac(tree, table1, 'sample1', 'sample3')

0.506024096385542

In [16]:
unweighted_unifrac(tree, table1, 'sample2', 'sample3')

0.8089887640449439

Finally, we can define a function to compute all pairs of unweighted UniFrac distances between our samples and return an [skbio.DistanceMatrix object](http://scikit-bio.org/docs/latest/generated/generated/skbio.stats.distance.DistanceMatrix.html#skbio.stats.distance.DistanceMatrix). Notice that the diagonal in this matrix is zero (because the distance between a sample and itself is always zero), and that the matrix is symmetric (because the distance between *sample A* and *sample B* is always equal to the distance between *sample B* and *sample A*).

In [17]:
from skbio import DistanceMatrix

def unweighted_unifrac_all(tree, table):
    sample_ids = table.columns
    num_samples = len(sample_ids)
    result = np.zeros((num_samples, num_samples))
    for i in range(num_samples):
        for j in range(i):
            result[i, j] = result[j, i] = unweighted_unifrac(tree, table, sample_ids[i], sample_ids[j])
    return DistanceMatrix(result, sample_ids)

In [18]:
dm = unweighted_unifrac_all(tree, table1)
print(dm)

3x3 distance matrix
IDs:
sample1, sample2, sample3
Data:
[[ 0.          0.4109589   0.5060241 ]
 [ 0.4109589   0.          0.80898876]
 [ 0.5060241   0.80898876  0.        ]]
